**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
!pip install scikit-video

    100% |████████████████████████████████| 2.3MB 13.4MB/s 


In [3]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act aims to select the next action.
During training, the agent uses $\epsilon$-strategy for exploration, i.e. it chooses a random action with probability $\epsilon$, or the learned action with probability 1-$\epsilon$. 

The value of $\epsilon$ influences the speed of convergence. The algorithm may find the optimal policy faster with a larger $\epsilon$, however, if the value of $\epsilon$ is too large, the algorithm may not be able to converge.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epsilon=0.1
epochs_train=300 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` is to record the current position of the rat. If the coordinate of the rat is [x, y], then position[x, y] = 1.

```board``` is to record the current states of the cells. If the rat has already visited a cell [x, y], then there will be no more reward in this cell, i.e. board[x, y] = 0.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)
            # action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=False)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/17.0. Average score (-8.0)
Win/lose count 14.5/10.0. Average score (-1.75)
Win/lose count 12.5/9.0. Average score (0.0)
Win/lose count 10.0/10.0. Average score (0.0)
Win/lose count 8.5/12.0. Average score (-0.7)
Win/lose count 11.0/14.0. Average score (-1.0833333333333333)
Win/lose count 10.5/11.0. Average score (-1.0)
Win/lose count 9.5/5.0. Average score (-0.3125)
Win/lose count 3.0/14.0. Average score (-1.5)
Win/lose count 5.0/12.0. Average score (-2.05)
Win/lose count 10.5/22.0. Average score (-2.909090909090909)
Win/lose count 7.5/14.0. Average score (-3.2083333333333335)
Win/lose count 6.5/12.0. Average score (-3.3846153846153846)
Win/lose count 12.5/12.0. Average score (-3.107142857142857)
Win/lose count 14.0/13.0. Average score (-2.8333333333333335)
Win/lose count 8.5/12.0. Average score (-2.875)
Win/lose count 7.0/15.0. Average score (-3.176470588235294)
Win/lose count 10.5/14.0. Average score (-3.1944444444444446)
Win/lose count 10.5/12.0. Average score (-3

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Proof.

Part I.
\begin{equation*}
\begin{split}
Q^\pi(s,a) & = E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
 & = r(s,a)+E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
 & = r(s,a)+\gamma\sum_{s',a'}p(s_1=s',a_1=a'|s_0=s,a_0=a)E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t-1}r(s_t,a_t)|s_1=s',a_1=a'] \\
 & = r(s,a)+\gamma\sum_{s',a'}p(s_1=s',a_1=a'|s_0=s,a_0=a)Q^{\pi}(s',a') \\
 & = r(s,a)+\gamma E_{(s',a')\sim p(.|s,a)}Q^{\pi}(s',a')\\
 & = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')].
\end{split}
\end{equation*}

Part II.

Let $\pi=(a', \pi')$,

\begin{equation*}
\begin{split}
Q^*(s,a) &= \max_{\pi}Q^{\pi}(s,a) \\
 &= \max_{\pi}[r(s,a)+\gamma E_{(s',a')\sim p(.|s,a)}Q^{\pi}(s',a')] \\
 &= r(s,a)+\gamma \max_{\pi}E_{(s',a')\sim p(.|s,a)}Q^{\pi}(s',a') \\
 &= r(s,a)+\gamma \max_{a',\pi'}\sum_{s'}p(s'|s_0=s,a_0=a)Q^{\pi'}(s',a') \\
 &= r(s,a)+\gamma \max_{a'}\sum_{s'}p(s'|s_0=s,a_0=a)\max_{\pi'}Q^{\pi'}(s',a') \\
 &= r(s,a)+\gamma \max_{a'}\sum_{s'}p(s'|s_0=s,a_0=a)Q^{*}(s',a') \\
 &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^*(s',a').
\end{split}
\end{equation*}

Part III.

The loss function is defined by the expected difference between the maximum possible Q-value from the next state<br/>
$$Q_{target}=r+\gamma\max_{a'}Q(s',a',\theta),$$
and current prediction $Q(s,a,\theta)$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m) 
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.choice(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | Average score ({})"
              .format(e, epoch, loss, win, lose, win-lose, score/(1+e)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1,5,5,self.n_state))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(s_.reshape(1,5,5,self.n_state))[0]
            if game_over_:
                ######## FILL IN
                target_q[i][a_] = r_
            else:
                ######## FILL IN
                target_q[i][a_] = r_ + self.discount*np.max(self.model.predict(n_s_.reshape(1,5,5,self.n_state))[0])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(50))
        model.add(Activation('relu'))
        model.add(Dense(50))
        model.add(Activation('relu'))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
epochs_train = 300
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, lr=.1, epsilon = epsilon, memory_size=500, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train290.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/300 | Loss 0.0043 | Win/lose count 3.0/3.0 (0.0) | Average score (0.0)
Epoch 001/300 | Loss 0.0012 | Win/lose count 3.0/5.0 (-2.0) | Average score (-1.0)
Epoch 002/300 | Loss 0.0155 | Win/lose count 5.0/7.0 (-2.0) | Average score (-1.3333333333333333)
Epoch 003/300 | Loss 0.0129 | Win/lose count 1.5/7.0 (-5.5) | Average score (-2.375)
Epoch 004/300 | Loss 0.0111 | Win/lose count 3.5/1.0 (2.5) | Average score (-1.4)
Epoch 005/300 | Loss 0.0032 | Win/lose count 3.5/0 (3.5) | Average score (-0.5833333333333334)
Epoch 006/300 | Loss 0.0081 | Win/lose count 4.0/8.0 (-4.0) | Average score (-1.0714285714285714)
Epoch 007/300 | Loss 0.0042 | Win/lose count 3.5/2.0 (1.5) | Average score (-0.75)
Epoch 008/300 | Loss 0.0058 | Win/lose count 3.0/2.0 (1.0) | Average score (-0.5555555555555556)
Epoch 009/300 | Loss 0.0073 | Win/lose count 3.0/2.0 (1.0) | Average score (-0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(8,3,input_shape=(5, 5, self.n_state), activation='relu', name='conv1'))
#         model.add(MaxPooling2D(pool_size=(2, 2), strides=1, name="pooling_1"))
        model.add(Conv2D(16,3, activation='relu', name='conv2'))
#         model.add(MaxPooling2D(pool_size=(2, 2), strides=1, name="pooling_2"))
        model.add(Flatten(name="flatten"))
        model.add(Dense(4, name="dense_1"))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train90.mp4'))

Epoch 000/300 | Loss 0.0054 | Win/lose count 2.0/4.0 (-2.0) | Average score (-2.0)
Epoch 001/300 | Loss 0.0146 | Win/lose count 1.5/4.0 (-2.5) | Average score (-2.25)
Epoch 002/300 | Loss 0.0060 | Win/lose count 1.5/4.0 (-2.5) | Average score (-2.3333333333333335)
Epoch 003/300 | Loss 0.0071 | Win/lose count 2.0/6.0 (-4.0) | Average score (-2.75)
Epoch 004/300 | Loss 0.0173 | Win/lose count 1.5/4.0 (-2.5) | Average score (-2.7)
Epoch 005/300 | Loss 0.0027 | Win/lose count 3.5/2.0 (1.5) | Average score (-2.0)
Epoch 006/300 | Loss 0.0054 | Win/lose count 3.0/5.0 (-2.0) | Average score (-2.0)
Epoch 007/300 | Loss 0.0175 | Win/lose count 6.5/4.0 (2.5) | Average score (-1.4375)
Epoch 008/300 | Loss 0.0051 | Win/lose count 5.0/4.0 (1.0) | Average score (-1.1666666666666667)
Epoch 009/300 | Loss 0.0055 | Win/lose count 5.0/2.0 (3.0) | Average score (-0.75)
Epoch 010/300 | Loss 0.0075 | Win/lose count 4.5/5.0 (-0.5) | Average score (-0.7272727272727273)
Epoch 011/300 | Loss 0.0060 | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

When the temperature is 0.3, the final scores of DQN_CNN and DQN_FC are:7.075 and 4.855 (average performance in 100 epochs). Therefore, we can conclude that DQN_CNN has a better performance than DQN_FC. By observing the behaviors displayed in the videos, we see that the algorithms tend to keep visiting the visited cells, i.e., they are not able to sufficiently explore the map.

By trying different temperatures, we find that a higher temperature results in a better performance.

| Temperature | DQN_CNN | DQN_FC
|------|------|------|
| 0.1 | 2.85 | 1.735 |
| 0.3 | 7.075 | 4.855 |
| 0.5 | 14.205 | 10.745 |

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.5/0. Average score (7.5)
Win/lose count 4.0/1.0. Average score (5.25)
Win/lose count 7.0/0. Average score (5.833333333333333)
Win/lose count 0/0. Average score (4.375)
Win/lose count 9.5/0. Average score (5.4)
Win/lose count 15.0/0. Average score (7.0)
Win/lose count 11.0/0. Average score (7.571428571428571)
Win/lose count 10.5/0. Average score (7.9375)
Win/lose count 8.0/0. Average score (7.944444444444445)
Win/lose count 0.5/1.0. Average score (7.1)
Win/lose count 5.5/0. Average score (6.954545454545454)
Win/lose count 12.5/0. Average score (7.416666666666667)
Win/lose count 6.0/0. Average score (7.3076923076923075)
Win/lose count 5.5/0. Average score (7.178571428571429)
Win/lose count 24.5/0. Average score (8.333333333333334)
Win/lose count 8.0/0. Average score (8.3125)
Win/lose count 9.5/0. Average score (8.382352941176471)
Win/lose count 6.5/1.0. Average score (8.222222222222221)
Win/lose count 4.0/0. Average score (8.0)
Win/lose count 2.5/0. Avera

In [0]:
HTML(display_videos('cnn_test80.mp4'))

In [0]:
HTML(display_videos('fc_test80.mp4'))

### try different temperatures.

In [0]:
temperature = 0.1
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 6.5/0. Average score (3.5)
Win/lose count 1.5/0. Average score (2.8333333333333335)
Win/lose count 0/1.0. Average score (1.875)
Win/lose count 4.5/0. Average score (2.4)
Win/lose count 0.5/0. Average score (2.0833333333333335)
Win/lose count 1.0/0. Average score (1.9285714285714286)
Win/lose count 3.0/0. Average score (2.0625)
Win/lose count 0.5/0. Average score (1.8888888888888888)
Win/lose count 1.5/0. Average score (1.85)
Win/lose count 4.0/0. Average score (2.0454545454545454)
Win/lose count 2.0/0. Average score (2.0416666666666665)
Win/lose count 1.5/0. Average score (2.0)
Win/lose count 3.0/0. Average score (2.0714285714285716)
Win/lose count 2.5/0. Average score (2.1)
Win/lose count 3.0/0. Average score (2.15625)
Win/lose count 1.0/0. Average score (2.088235294117647)
Win/lose count 6.0/0. Average score (2.3055555555555554)
Win/lose count 1.5/0. Average score (2.263157894736842)
Win/lose count 0.5/0. Averag

In [0]:
temperature = 0.5
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = epsilon, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/0. Average score (5.0)
Win/lose count 21.5/1.0. Average score (12.75)
Win/lose count 8.0/0. Average score (11.166666666666666)
Win/lose count 24.5/0. Average score (14.5)
Win/lose count 28.5/0. Average score (17.3)
Win/lose count 15.5/0. Average score (17.0)
Win/lose count 2.5/1.0. Average score (14.785714285714286)
Win/lose count 2.5/0. Average score (13.25)
Win/lose count 4.5/0. Average score (12.277777777777779)
Win/lose count 16.0/1.0. Average score (12.55)
Win/lose count 11.5/0. Average score (12.454545454545455)
Win/lose count 25.0/0. Average score (13.5)
Win/lose count 15.5/0. Average score (13.653846153846153)
Win/lose count 8.5/0. Average score (13.285714285714286)
Win/lose count 26.5/2.0. Average score (14.033333333333333)
Win/lose count 24.5/0. Average score (14.6875)
Win/lose count 6.5/0. Average score (14.205882352941176)
Win/lose count 33.0/0. Average score (15.25)
Win/lose count 9.0/0. Average score (14.921052631578947)
Win/lose count 1

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
explore_start = 1.0            # exploration probability at start
explore_stop = 0.1            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

def train_explore(agent,env,epoch,explore_start,explore_stop,decay_rate,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    decay_step = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # agent.set_epsilon(0.5)
        
        while not game_over:
            decay_step += 1
            agent.set_epsilon(explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step))
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.1f}) | Average score ({}) | Explore P {:.4f} "
              .format(e, epoch, loss, win, lose, win-lose, score/(1+e), agent.epsilon))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # malus_position adds a penalty to already visited cells, the penalty increases 0.1 each time the cell is visited again
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1

        reward = reward + self.board[self.x, self.y]
        
        # reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                            self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                        self.position.reshape(self.grid_size, self.grid_size, 1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [16]:
# Training
epochs_train = 300
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
# env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=3)
# agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=2)
train_explore(agent, env, epochs_train, explore_start, explore_stop, decay_rate, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore290.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/300 | Loss 0.0454 | Win/lose count 9.0/48.6 (-39.6) | Average score (-39.60000000000004) | Explore P 0.9821 
Epoch 001/300 | Loss 0.0299 | Win/lose count 12.0/41.8 (-29.8) | Average score (-34.70000000000003) | Explore P 0.9645 
Epoch 002/300 | Loss 0.0202 | Win/lose count 7.0/46.4 (-39.4) | Average score (-36.2666666666667) | Explore P 0.9473 
Epoch 003/300 | Loss 0.0382 | Win/lose count 7.5/48.2 (-40.7) | Average score (-37.37500000000004) | Explore P 0.9305 
Epoch 004/300 | Loss 0.0224 | Win/lose count 12.5/38.1 (-25.6) | Average score (-35.020000000000046) | Explore P 0.9139 
Epoch 005/300 | Loss 0.0208 | Win/lose count 8.0/49.0 (-41.0) | Average score (-36.01666666666671) | Explore P 0.8977 
Epoch 006/300 | Loss 0.0131 | Win/lose count 9.0/50.0 (-41.0) | Average score (-36.72857142857146) | Explore P 0.8819 
Epoch 007/300 | Loss 0.0222 | Win/lose count 

In [17]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 22.0/6.0. Average score (16.0)
Win/lose count 24.5/6.0. Average score (17.25)
Win/lose count 27.5/6.0. Average score (18.666666666666668)
Win/lose count 23.5/8.0. Average score (17.875)
Win/lose count 25.0/10.0. Average score (17.3)
Win/lose count 23.5/6.0. Average score (17.333333333333332)
Win/lose count 24.5/4.0. Average score (17.785714285714285)
Win/lose count 26.5/6.0. Average score (18.125)
Win/lose count 27.5/2.0. Average score (18.944444444444443)
Win/lose count 25.0/7.0. Average score (18.85)
Win/lose count 24.0/9.0. Average score (18.5)
Win/lose count 23.0/8.0. Average score (18.208333333333332)
Win/lose count 15.5/7.0. Average score (17.46153846153846)
Win/lose count 28.5/4.0. Average score (17.964285714285715)
Win/lose count 25.0/3.0. Average score (18.233333333333334)
Win/lose count 20.5/4.0. Average score (18.125)
Win/lose count 21.0/6.0. Average score (17.941176470588236)
Win/lose count 28.5/3.0. Average score (18.36111111111111)
Win/lose count 21.0/11.0.

We can see that the exploration problem is solved and the final scores are much improved by using the decreasing  𝜖 -greedy exploration and adding a new state.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
# this function is used to collect (state, action) pairs generated by our expert DQN.
def collect_pair(agent,env,epochs):
    # Number of won games
    score = 0
    states = []
    actions = []
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)
            # action = agent.act(state)
            states.append(state)
            actions.append(action)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=False)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    return states, actions

In [20]:
# use the expert DQN to generate winning games
states, actions = collect_pair(agent,env,20)

Win/lose count 29.0/3.0. Average score (26.0)
Win/lose count 23.5/5.0. Average score (22.25)
Win/lose count 23.5/6.0. Average score (20.666666666666668)
Win/lose count 21.5/2.0. Average score (20.375)
Win/lose count 26.0/6.0. Average score (20.3)
Win/lose count 23.0/9.0. Average score (19.25)
Win/lose count 22.5/10.0. Average score (18.285714285714285)
Win/lose count 22.0/5.0. Average score (18.125)
Win/lose count 21.5/3.0. Average score (18.166666666666668)
Win/lose count 22.5/11.0. Average score (17.5)
Win/lose count 24.5/8.0. Average score (17.40909090909091)
Win/lose count 22.0/4.0. Average score (17.458333333333332)
Win/lose count 21.0/8.0. Average score (17.115384615384617)
Win/lose count 25.5/3.0. Average score (17.5)
Win/lose count 19.5/3.0. Average score (17.433333333333334)
Win/lose count 24.5/7.0. Average score (17.4375)
Win/lose count 26.5/7.0. Average score (17.558823529411764)
Win/lose count 26.0/5.0. Average score (17.75)
Win/lose count 25.5/9.0. Average score (17.684210

In [38]:
# train a model that mimicks the expert's behavior
model = Sequential()
model.add(Conv2D(8,3,input_shape=(5, 5, 3), activation='relu', name='conv1'))
#model.add(MaxPooling2D(pool_size=(2, 2), padding='same', name="pooling_1"))
model.add(Conv2D(16,3, activation='relu', name='conv2'))
#model.add(MaxPooling2D(pool_size=(2, 2), padding='same', name="pooling_2"))
model.add(Flatten(name="flatten"))
model.add(Dense(4, name="dense_1", activation='softmax')) 
model.compile(sgd(lr=0.1, decay=1e-4, momentum=0.0), "categorical_crossentropy")
model.fit(np.array(states), np.eye(4)[actions], epochs=20)

Epoch 1/20
4020/4020 [==============================] - 1s 213us/step - loss: 1.2973
Epoch 2/20
4020/4020 [==============================] - 0s 108us/step - loss: 0.9807
Epoch 3/20
4020/4020 [==============================] - 0s 112us/step - loss: 0.8366
Epoch 4/20
4020/4020 [==============================] - 0s 110us/step - loss: 0.7731
Epoch 5/20
4020/4020 [==============================] - 0s 117us/step - loss: 0.6756
Epoch 6/20
4020/4020 [==============================] - 0s 118us/step - loss: 0.5952
Epoch 7/20
4020/4020 [==============================] - 0s 121us/step - loss: 0.5230
Epoch 8/20
4020/4020 [==============================] - 0s 122us/step - loss: 0.4789
Epoch 9/20
4020/4020 [==============================] - 0s 120us/step - loss: 0.4580
Epoch 10/20
4020/4020 [==============================] - 0s 116us/step - loss: 0.4315
Epoch 11/20
4020/4020 [==============================] - 0s 122us/step - loss: 0.4168
Epoch 12/20
4020/4020 [==============================] - 0s 118

In [0]:
# this function is used to test the performance of our imitation model.
def test_imit(model,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            # action = agent.act(state, train=False)
            action = np.argmax(model.predict(state.reshape(1,5,5,3)), axis=1)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [39]:
test_imit(model,env,50,prefix='imitation')

Win/lose count 19.5/17.09999999999998. Average score (2.40000000000002)
Win/lose count 24.5/13.29999999999999. Average score (6.800000000000015)
Win/lose count 31.0/15.999999999999977. Average score (9.533333333333351)
Win/lose count 26.5/17.400000000000002. Average score (9.425000000000011)
Win/lose count 26.0/20.69999999999999. Average score (8.600000000000012)
Win/lose count 23.0/14.799999999999986. Average score (8.533333333333346)
Win/lose count 24.0/14.199999999999987. Average score (8.714285714285726)
Win/lose count 23.5/10.599999999999989. Average score (9.237500000000011)
Win/lose count 24.5/14.29999999999998. Average score (9.344444444444456)
Win/lose count 22.5/12.999999999999982. Average score (9.360000000000012)
Win/lose count 24.0/17.499999999999993. Average score (9.100000000000012)
Win/lose count 21.5/19.3. Average score (8.525000000000011)
Win/lose count 24.5/12.899999999999988. Average score (8.761538461538473)
Win/lose count 22.5/17.499999999999986. Average score (8.

We can see that the imitation model (score: 7.13) does not perform as well as the DQN (score: 18.125).

***